In [86]:

import calendar
from scipy.stats import chi2_contingency
months = list(map(lambda x: x.lower(), calendar.month_name))[1:]
import pandas as pd
import statsmodels.api as sm
from pandas import DataFrame

import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = 'all'

import matplotlib.pylab as pylab

params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 10),
          'axes.labelsize': 'x-large',
          'axes.titlesize': 'x-large',
          'xtick.labelsize': 'x-large',
          'ytick.labelsize': 'x-large'}
pylab.rcParams.update(params)

import os

project_folder = f'{os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))}/data/process'
import warnings

warnings.filterwarnings(action='ignore')
from IPython.display import display
from IPython.display import Markdown
from IPython.display import display_html


questions ={'Q2':'part','Q6':'day_time','Q7':'is_locked','Q8':'locked_type','Q9':'theft_loc','Q10':'is_registered','Q11':'where_rep','Q12':'is_insured','Q13':'value_approx','Q14':'is_electric','Q15':'bicycle_type','Q17':'is_univ','Q18':'is_recover','Q19':'is_online','Q20':'how_recover','Q21':'is_police','Q23':'recovery_loc','Q24':'rec_cond','Q25':'is_replaced', 'Q26_new':'days_to_bikes', 'Q28':'seasons', 'Q29':'purpose', 'Q30':'mode_alt', 'Q31':'post_act','Q32':'why_less','age_groups': 'age', 'Q35': 'gender', 'Q36': 'income','Q37':'numb_bikes ', 'Q38': 'education', 'Q39': 'ethnic_origin','country':'country','city':'city'}

In [87]:

data = pd.read_csv(f'{project_folder}/new_data/new_data_2.csv')
data.columns

Index(['Unnamed: 0.3', 'index', 'Q26', 'level_0', 'Unnamed: 0.2',
       'Unnamed: 0.1', 'Unnamed: 0', 'StartDate', 'EndDate', 'Status',
       'IPAddress', 'Progress', 'Duration (in seconds)', 'Finished',
       'RecordedDate', 'ResponseId', 'RecipientLastName', 'RecipientFirstName',
       'RecipientEmail', 'ExternalReference', 'LocationLatitude',
       'LocationLongitude', 'DistributionChannel', 'UserLanguage',
       'Introduction', 'Q1', 'Q2', 'Q3', 'Q3_6_TEXT', 'Q4', 'Q5', 'Q6', 'Q7',
       'Q8', 'Q8_9_TEXT', 'Q9', 'Q9_11_TEXT', 'Q10', 'Q11', 'Q11_4_TEXT',
       'Q12', 'Q13', 'Q14', 'Q15', 'Q16', 'Q16_1_TEXT', 'Q17', 'Q18', 'Q19',
       'Q19_6_TEXT', 'Q20', 'Q20_4_TEXT', 'Q21', 'Q22', 'Q22_1_TEXT', 'Q23',
       'Q23_15_TEXT', 'Q24', 'Q25', 'Q27', 'Q28_1', 'Q28_2', 'Q28_3', 'Q28_4',
       'Q29', 'Q30', 'Q31', 'Q32', 'Q32_10_TEXT', 'Q33', 'Q33_9_TEXT', 'Q34',
       'Q35', 'Q36', 'Q37', 'Q38', 'Q39', 'Q39_10_TEXT', 'Q40', 'Q40_3_TEXT',
       'lat', 'lon', 'score', 'stolen_bi

In [88]:
#
def calculate_chi_cross_tab(two_var_tables,var1,var2):
    r"""
    This function creates new groups based on the data in @vars, cross tab them and calculate Chi-square
    :return:
    """

    two_var_tables[[var1['var'],var2['var']]] = two_var_tables.apply(lambda x: (x[var1['q']] if x[var1['q']] in var1['group'] else var1['group2'],x[var2['q']] if x[var2['q']] in var2['group'] else var2['group2']),axis=1,result_type='expand')

    cont_table = pd.crosstab(two_var_tables[var1['var']], two_var_tables[var2['var']],normalize='columns')*100

    # perform chi-square test of independence
    chi2, p, dof, expected = chi2_contingency(cont_table)

    # print the test results
    print(f"Chi-square statistic: {chi2:.2f}")
    print(f"P-value: {p:.5f}")
    print(cont_table)

In [113]:
def prep_for_chi(only_rel_f):
    r"""
    this function store and set the data so only the relevant data will be used to calculate chi -square
    :param only_rel_f:
    :return:
    """
    # working only with information stored in list in @rel_data for a specific col
    if rel_data[1]:
        only_rel_f= only_rel_f[only_rel_f[rel_data[0][0]].isin(rel_data[1])] if rel_data[0][1] else only_rel_f[~only_rel_f[rel_data[0][0]].isin(rel_data[1])]
    if rel_data[3]:
        only_rel_f= only_rel_f[only_rel_f[rel_data[2][0]].isin(rel_data[3])] if rel_data[2][1] else only_rel_f[~only_rel_f[rel_data[2][0]].isin(rel_data[3])]

    calculate_chi_cross_tab(only_rel_f,var0,var1)

In [125]:
def explore_date(is_count_f,cross_tab= False):
    r"""
    Clean the data and print cross_tab if it is required
    :param is_count_f:
    :param cross_tab:
    :return:
    """
    only_rel_f= data[cols_name].dropna()
    for q_temp in cols_name:
        if q_temp in is_count_f:
            only_rel_f[q_temp]= only_rel_f[q_temp].str.replace(r"\(.*?\)", "").str.replace(":", "").str.strip().str.count(',')
        else:
            only_rel_f[q_temp]= only_rel_f[q_temp].str.replace(r"\(.*?\)", "").str.replace(":", "").str.strip()
        print(only_rel_f[q_temp].value_counts())
    if cross_tab:
        print(pd.crosstab(only_rel_f[cols_name[0]], only_rel_f[cols_name[1]],normalize='columns')*100)
    return only_rel_f

In [134]:
# ,'Q24':'rec_cond' and 'Q15':'bicycle_type'
# Parameters
q_0 = 'Q24'
q_1 = 'Q15'
cols_name = [q_0,q_1]
calc_chi = False
is_count_loc= []

only_rel =explore_date(is_count_loc,True)
# Parameter for chi-squared
# The second argument in the nested tuple determine if the data in the following list is the filtered or the selected one
rel_data = ((q_0,True),['Yes, on Kijiji','Yes, on Facebook Marketplace'],(q_1,True),[])
var0 = {'q':q_0,'var':questions[q_0],'group':['Yes, on Kijiji'],'group2': 'Yes, on Facebook Marketplace'}
var1 = {'q':q_1,'var':questions[q_1],'group':['USA'],'group2':'CAN'}

if calc_chi:
    prep_for_chi(only_rel)

Rideable– bicycle was safe to operate and immediately usable                       63
Repairable– bicycle was damaged and repairs were required before being rideable    45
Salvage/junk– bicycle was not rideable or repairable                                8
Don’t know/not sure                                                                 3
Name: Q24, dtype: int64
Mountain                   43
Hybrid/City/Dutch          30
Road                       22
Gravel/cyclocross           7
Delivery/Cargo              7
Cruiser                     4
Folding                     2
Recumbent                   1
BMX                         1
Tricycle or quadricycle     1
Fixed gear                  1
Name: Q15, dtype: int64
Q15                                                   BMX  Cruiser  \
Q24                                                                  
Don’t know/not sure                                   0.0      0.0   
Repairable– bicycle was damaged and repairs wer...    0.0     75.0  

In [132]:
# is_online and country
# Parameters
q_0 = 'Q19'
q_1 = 'country'
cols_name = [q_0,q_1]
calc_chi = True
is_count_loc= []

only_rel =explore_date(is_count_loc,False)
# Parameter for chi-squared
# The second argument in the nested tuple determine if the data in the following list is the filtered or the selected one
rel_data = ((q_0,True),['Yes, on Kijiji','Yes, on Facebook Marketplace'],(q_1,True),[])
var0 = {'q':q_0,'var':questions[q_0],'group':['Yes, on Kijiji'],'group2': 'Yes, on Facebook Marketplace'}
var1 = {'q':q_1,'var':questions[q_1],'group':['USA'],'group2':'CAN'}

if calc_chi:
    prep_for_chi(only_rel)

​No                             166
Don’t know/not sure              37
Yes, on Facebook Marketplace     23
Yes, on OfferUp                  20
Yes, on Craigslist               15
Yes, on other website            13
Yes, on Kijiji                    6
Name: Q19, dtype: int64
USA    203
CAN     77
Name: country, dtype: int64
Chi-square statistic: 72.28
P-value: 0.00000
country                             CAN    USA
is_online                                     
Yes, on Facebook Marketplace  45.454545  100.0
Yes, on Kijiji                54.545455    0.0


In [129]:
# education  and how recover
# Parameters
q_0 = 'Q20'
q_1 = 'Q38'
cols_name = [q_0,q_1]
calc_chi = True
is_count_loc= []

only_rel =explore_date(is_count_loc,False)
# Parameter for chi-squared
# The second argument in the nested tuple determine if the data in the following list is the filtered or the selected one
rel_data = ((q_0,True),[],(q_1,True),['Bachelor’s degree','Graduate degree','Some university'])
var0 = {'q':q_0,'var':questions[q_0],'group':['​I found and recovered it myself'],'group2': 'Other'}
var1 = {'q':q_1,'var':questions[q_1],'group':['Bachelor’s degree','Graduate degree'],'group2':'Some university'}

if calc_chi:
    prep_for_chi(only_rel)

The police found it and returned it to me​                                    91
I recovered it with online support                                            53
​I found and recovered it myself                                              51
Other                                                                         45
A member of the public/friend/family member found it and returned it to me    28
Name: Q20, dtype: int64
Bachelor’s degree                          105
Graduate degree                            104
Some university                             27
Associate’s/vocational/technical degree     19
Graduated high school                        7
I prefer to not answer                       4
Some high school or less                     2
Name: Q38, dtype: int64
Chi-square statistic: 9.47
P-value: 0.00879
education                         Bachelor’s degree  Graduate degree  \
how_recover                                                            
Other                         

In [119]:
# gender and how recover
# Parameters
q_0 = 'Q20'
q_1 = 'Q35'
cols_name = [q_0,q_1]
calc_chi = True
is_count_loc= []

# Parameter for chi-squared
# The second argument in the nested tuple determine if the data in the following list is the filtered or the selected one
rel_data = ((q_0,True),['The police found it and returned it to me​','​I found and recovered it myself','I recovered it with online support'],(q_1,True),['Man','Woman'])
only_rel =explore_date(is_count_loc)


# More parameters
var0 = {'q':q_0,'var':questions[q_0],'group':['The police found it and returned it to me​','​I found and recovered it myself'],'group2': 'I recovered it with online support'}
var1 = {'q':q_1,'var':questions[q_1],'group':['Man'],'group2':'Woman'}

if calc_chi:
    prep_for_chi(only_rel)

The police found it and returned it to me​                                    92
I recovered it with online support                                            53
​I found and recovered it myself                                              51
Other                                                                         45
A member of the public/friend/family member found it and returned it to me    28
Name: Q20, dtype: int64
Man                          165
Woman                         99
Prefer not to say              3
Non-binary / third gender      2
Name: Q35, dtype: int64
Chi-square statistic: 1.62
P-value: 0.44554
gender                                            Man      Woman
how_recover                                                     
I recovered it with online support          23.529412  31.506849
The police found it and returned it to me​  49.579832  45.205479
​I found and recovered it myself            26.890756  23.287671


In [115]:
# value_approximation and how recover
# Parameters
q_0 = 'Q20'
q_1 = 'Q13'
cols_name = [q_0,q_1]
calc_chi = True
is_count_loc= []

# Parameter for chi-squared
# The second argument in the nested tuple determine if the data in the following list is the filtered or the selected one
rel_data = ((q_0,True),[],(q_1,False),[])
only_rel =explore_date(is_count_loc)

# Group the data
# In order to execute grouping we should first eliminate irrelevant information
only_rel = only_rel[only_rel[q_1]!='Don’t know/not sure']

df = DataFrame(only_rel[q_1].unique()).dropna().sort_values(by=0,ascending=0).reset_index(drop=True)
sorted_list = pd.concat([df.loc[0], df.loc[4], df.loc[2],df.loc[6],df.loc[5],df.iloc[3],df.loc[1]])[0].to_list()
only_rel[q_1] = only_rel[q_1].apply(lambda x:1 if sorted_list.index(x)>3 else 0)

# More parameters
var0 = {'q':q_0,'var':questions[q_0],'group':['​I found and recovered it myself'],'group2':'Other'}
var1 = {'q':q_1,'var':questions[q_1],'group':[0],'group2': 1}

if calc_chi:
    prep_for_chi(only_rel)

The police found it and returned it to me​                                    96
I recovered it with online support                                            57
​I found and recovered it myself                                              53
Other                                                                         45
A member of the public/friend/family member found it and returned it to me    28
Name: Q20, dtype: int64
$2000-3999             67
$1000-$1999            66
$500-$999              57
$250-$499              37
$4000-6999             28
$7000 or more          15
Less than $250          8
Don’t know/not sure     1
Name: Q13, dtype: int64
Chi-square statistic: 6.31
P-value: 0.01202
value_approx                              0     1
how_recover                                      
Other                             75.595238  90.0
​I found and recovered it myself  24.404762  10.0


In [116]:
# register and how recover
# Parameters
q_0 = 'Q20'
q_1 = 'Q10'
cols_name = [q_0,q_1]
calc_chi = True
is_count= []
# Parameter for chi-squared
rel_data = (q_0,['The police found it and returned it to me​','I recovered it with online support'],q_1,['Yes','No'])

# Explore the data
only_rel= data[cols_name].dropna()
for q in cols_name:
    if q in is_count:
        only_rel[q]= only_rel[q].str.replace(r"\(.*?\)","").str.replace(":", "").str.strip().str.count(',')
    else:
        only_rel[q]= only_rel[q].str.replace(r"\(.*?\)","").str.replace(":", "").str.strip()
    print(only_rel[q].value_counts())

if calc_chi:
    var_0 = {'q':q_0,'var':questions[q_0],'group':['The police found it and returned it to me​'],'group2':'I recovered it with online support'}
    var_1 = {'q':q_1,'var':questions[q_1],'group':['Yes'],'group2': 'No'}
    # working only with information stored in list in @rel_data for a specific col
    if rel_data[1]:
        only_rel= only_rel[only_rel[rel_data[0]].isin(rel_data[1])]
    if rel_data[3]:
        only_rel= only_rel[only_rel[rel_data[2]].isin(rel_data[3])]

    calculate_chi_cross_tab(only_rel,var_0,var_1)

The police found it and returned it to me​                                    96
I recovered it with online support                                            57
​I found and recovered it myself                                              53
Other                                                                         45
A member of the public/friend/family member found it and returned it to me    28
Name: Q20, dtype: int64
No                     154
Yes                    120
Don’t know/not sure      5
Name: Q10, dtype: int64
Chi-square statistic: 3.55
P-value: 0.05969
is_registered                                      No        Yes
how_recover                                                     
I recovered it with online support          43.023256  29.230769
The police found it and returned it to me​  56.976744  70.769231


In [73]:
# country and is police
# Parameters
q_0 = 'Q21'
q_1 = 'country'
cols_name = [q_0,q_1]
calc_chi = True
is_count= []
rel_data = (q_1,[])

# Explore the data
only_rel= data[cols_name].dropna()
for q in cols_name:
    if q in is_count:
        only_rel[q]= only_rel[q].str.replace(r"\(.*?\)","").str.replace(":", "").str.strip().str.count(',')
        # specific code to this analysis
        only_rel[only_rel[q_1]==3] = 2
    else:
        only_rel[q]= only_rel[q].str.replace(r"\(.*?\)","").str.replace(":", "").str.strip()
    print(only_rel[q].value_counts())

if calc_chi:
    var_0 = {'q':q_0,'var':questions[q_0],'group':["Yes, in-person support",'Yes, phone or email support'],'group2': 'No'}
    var_1 = {'q':q_1,'var':questions[q_1],'group':['USA'],'group2':'CAN'}
    # working only with information stored in list in @rel_data for a specific col
    if rel_data[1]:
        only_rel= only_rel[only_rel[rel_data[0]].isin(rel_data[1])]

    calculate_chi_cross_tab(only_rel,var_0,var_1)

Yes, in-person support         109
​No                            103
Yes, phone or email support     68
Name: Q21, dtype: int64
USA    203
CAN     77
Name: country, dtype: int64
Chi-square statistic: 20.37
P-value: 0.00004
country                            CAN        USA
is_police                                        
No                           15.584416  44.827586
Yes, in-person support       50.649351  34.482759
Yes, phone or email support  33.766234  20.689655


In [68]:
# country and how recover
# Parameters
q_0 = 'Q20'
q_1 = 'country'
cols_name = [q_0,q_1]
calc_chi = True
is_count= []
rel_data = (q_1,[])

# Explore the data
only_rel= data[cols_name].dropna()
for q in cols_name:
    if q in is_count:
        only_rel[q]= only_rel[q].str.replace(r"\(.*?\)","").str.replace(":", "").str.strip().str.count(',')
        # specific code to this analysis
        only_rel[only_rel[q_1]==3] = 2
    else:
        only_rel[q]= only_rel[q].str.replace(r"\(.*?\)","").str.replace(":", "").str.strip()
    print(only_rel[q].value_counts())

if calc_chi:
    var_0 = {'q':q_0,'var':questions[q_0],'group':["The police found it and returned it to me​"],'group2': 'other'}
    var_1 = {'q':q_1,'var':questions[q_1],'group':['USA'],'group2':'CAN'}
    # working only with information stored in list in @rel_data for a specific col
    if rel_data[1]:
        only_rel= only_rel[only_rel[rel_data[0]].isin(rel_data[1])]

    calculate_chi_cross_tab(only_rel,var_0,var_1)

The police found it and returned it to me​                                    96
I recovered it with online support                                            57
​I found and recovered it myself                                              53
Other                                                                         45
A member of the public/friend/family member found it and returned it to me    28
Name: Q20, dtype: int64
USA    203
CAN     76
Name: country, dtype: int64
Chi-square statistic: 22.45
P-value: 0.00000
country                                           CAN        USA
how_recover                                                     
The police found it and returned it to me​  59.210526  25.123153
other                                       40.789474  74.876847


In [ ]:
# where_rep and is_recover by amount
# Parameters
q_0 = 'Q18'
q_1 = 'Q11'
cols_name = [q_0,q_1]
calc_chi = True
is_count= [q_1]
rel_data = (q_1,[])

# Explore the data
only_rel= data[cols_name].dropna()
for q in cols_name:
    if q in is_count:
        only_rel[q]= only_rel[q].str.replace(r"\(.*?\)","").str.replace(":", "").str.strip().str.count(',')
        # specific code to this analysis
        only_rel[only_rel[q_1]==3] = 2
    else:
        only_rel[q]= only_rel[q].str.replace(r"\(.*?\)","").str.replace(":", "").str.strip()
    print(only_rel[q].value_counts())

if calc_chi:
    var_0 = {'q':q_0,'var':questions[q_0],'group':["Yes"],'group2': 'No'}
    var_1 = {'q':q_1,'var':questions[q_1],'group':[0,1],'group2':2}
    # working only with information stored in list in @rel_data for a specific col
    if rel_data[1]:
        only_rel= only_rel[only_rel[rel_data[0]].isin(rel_data[1])]

    calculate_chi_cross_tab(only_rel,var_0,var_1)

In [51]:
# where_rep and is_recover

# Parameters
q_0 = 'Q18'
q_1 = 'country'
cols_name = [q_0,q_1]
calc_chi = True

rel_data = (q_1,['The police ,A bicycle registry/recovery system ,Social media','The police ,A bicycle registry/recovery system','The police','A bicycle registry/recovery system'])

# Explore the data
only_rel= data[cols_name].dropna()
for q in cols_name:
    only_rel[q]= only_rel[q].str.replace(r"\(.*?\)","").str.replace(":", "").str.strip()
    print(only_rel[q].value_counts())

if calc_chi:
    var_0 = {'q':q_0,'var':questions[q_0],'group':["Yes"],'group2': 'No'}
    var_1 = {'q':q_1,'var':questions[q_1],'group':rel_data[1][:-1],'group2': rel_data[1][-1]}
    # working only with information stored in list in @rel_data for a specific col
    if rel_data:
        only_rel= only_rel[only_rel[rel_data[0]].isin(rel_data[1])]

    calculate_chi_cross_tab(only_rel,var_0,var_1)


No     1535
Yes     280
Name: Q18, dtype: int64
The police ,A bicycle registry/recovery system ,Social media           618
The police ,A bicycle registry/recovery system                         533
The police                                                             304
A bicycle registry/recovery system                                      96
I did not report the theft                                              60
The police ,Social media                                                50
The police ,A bicycle registry/recovery system ,Social media ,Other     43
A bicycle registry/recovery system ,Social media                        37
The police ,A bicycle registry/recovery system ,Other                   21
Other                                                                   16
The police ,Other                                                       15
Social media                                                            10
The police ,Social media ,Other                     

In [31]:
# Time and is_recover
calc_chi = True
q_0 = 'Q18'
q_1 = 'Q11'
cols_name = [q_0,q_1]
# Explore the data
only_rel= data[cols_name].dropna()
for q in cols_name:
    only_rel[q]= only_rel[q].str.replace(r"\(.*?\)","").str.strip()
    print(only_rel[q].value_counts())

if calc_chi:
    var_0 = {'q':q_0,'var':questions[q_0],'group':["Yes"],'group2': 'No'}
    var_1 = {'q':q_1,'var':questions[q_1],'group':["Don’t know/not sure"],'group2': 'Other'}
    calculate_chi_cross_tab(only_rel,var_0,var_1)

No     1531
Yes     278
Name: Q18, dtype: int64
Overnight              734
Afternoon              354
Don’t know/not sure    328
Evening                223
Morning                170
Name: Q6, dtype: int64
Chi-square statistic: 1.93
P-value: 0.16453
day_time    Don’t know/not sure      Other
is_recover                                
No                    90.853659  83.254558
Yes                    9.146341  16.745442


In [26]:
# Country and is_recover
is_explore = False
q_0 = 'Q18'
q_1 = 'country'
cols_name = [q_0,q_1]
# Explore the data
if is_explore:
    only_rel= data[cols_name].dropna()
    for q in cols_name:
        only_rel[q]= only_rel[q].str.replace(r"\(.*?\)","").str.strip()
        print(only_rel[q].value_counts())
else:
    var_0 = {'q':q_0,'var':questions[q_0],'group':["Yes"],'group2': 'No'}
    var_1 = {'q':q_1,'var':questions[q_1],'group':["USA"],'group2': 'CAN'}
    calculate_chi_cross_tab(only_rel,var_0,var_1)

Chi-square statistic: 2.25
P-value: 0.13337
country           CAN        USA
is_recover                      
No          77.083333  86.293045
Yes         22.916667  13.706955


In [63]:
# Country and bike type
q_0 = 'Q15'
var_0 = {'q':q_0,'var':questions[q_0],'group':['Hybrid/City/Dutch ', 'Road',],'group2': 'Mountain  '}
var_1 = {'q':'country','var':'country','group':["USA"],'group2': 'CAN'}

only_rel= data[[var_0['q'],var_1['q']]].dropna()

clean_tables= only_rel[only_rel[var_0['q']].isin(['Hybrid/City/Dutch ', 'Road','Mountain  '])]

calculate_chi_cross_tab(clean_tables,var_0,var_1)

Chi-square statistic: 16.53
P-value: 0.00026
country                   CAN        USA
bicycle_type                            
Hybrid/City/Dutch   30.960854  38.696020
Mountain            58.007117  32.091448
Road                11.032028  29.212532


In [39]:
# Value and lock type


var_1 = {'q':'Q8','var':'locked_type','group':["Other (please specify):"],'group2': 'other'}
var_0 = {'q':'Q13','var':'value_approx','group':['$7000 or more '],'group2': 'Other'}

only_rel= data[[var_0['q'],var_1['q']]].dropna()
pd.crosstab(only_rel[var_1['q']],only_rel[var_0['q']],normalize='columns')*100
clean_tables_0 = only_rel[only_rel[var_1['q']].isin(['Cable lock ', '​U-lock      ','Other (please specify):'])]
clean_tables = clean_tables_0[~only_rel[var_0['q']].isin(['I prefer to not answer', 'Don’t know/not sure '])]
calculate_chi_cross_tab(clean_tables,var_1,var_0)

Q13,$1000-$1999,$2000-3999,$250-$499,$4000-6999,$500-$999,$7000 or more,Don’t know/not sure,I prefer to not answer,Less than $250
Q8,,,,,,,,,
Cable lock,30.487805,31.360947,37.414966,32.530120,37.345679,32.608696,33.333333,0.0,43.750000
"Cable lock ,Don’t know/not sure",0.000000,0.591716,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
"Cable lock ,Frame lock (aka wheel lock/O-lock/ring lock)",0.406504,0.000000,0.000000,0.000000,0.308642,2.173913,0.000000,0.0,0.000000
"Cable lock ,Other (please specify):",1.626016,1.183432,2.040816,2.409639,0.925926,4.347826,0.000000,0.0,0.000000
Chain-lock combination (chain with mini u-lock or integrated locking mechanism),5.691057,7.100592,13.605442,2.409639,8.333333,10.869565,33.333333,0.0,4.166667
"Chain-lock combination (chain with mini u-lock or integrated locking mechanism),Cable lock",0.813008,0.000000,0.680272,0.000000,0.308642,2.173913,0.000000,0.0,2.083333
"Chain-lock combination (chain with mini u-lock or integrated locking mechanism),Cable lock ,Other (please specify):",0.000000,0.000000,0.000000,0.000000,0.308642,0.000000,0.000000,0.0,0.000000
"Chain-lock combination (chain with mini u-lock or integrated locking mechanism),Chain-lock with padlock",0.000000,0.000000,0.000000,0.000000,0.308642,0.000000,0.000000,0.0,0.000000
"Chain-lock combination (chain with mini u-lock or integrated locking mechanism),Chain-lock with padlock ,Cable lock",0.000000,0.000000,0.000000,1.204819,0.000000,0.000000,0.000000,0.0,0.000000


In [33]:
# Value and part
# age and is_university

var_0 = {'q':'Q2','var':'part','group':["A major bicycle part necessary for riding (i.e., saddle, wheel, pedal, or handlebar)"],'group2': 'other'}
var_1 = {'q':'Q13','var':'value_approx','group':['Less than $250 '],'group2': 'Other'}
only_rel= data[[var_0['q'],var_1['q']]].dropna()
clean_tables = only_rel[~only_rel[var_1['q']].isin(['I prefer to not answer', 'Don’t know/not sure '])]

calculate_chi_cross_tab(clean_tables,var_0,var_1)

Chi-square statistic: 4.01
P-value: 0.04530
value_approx                                        Less than $250       Other
part                                                                          
A major bicycle part necessary for riding (i.e....         7.894737   1.044689
other                                                     92.105263  98.955311


In [22]:
# Gender and where to report
# age and is_university
num_to_report = 'num_to_report'
var_0 = {'q':'Q11','var':'where_rep','group':[2,3],'group2': 'other'}
var_1 = {'q':'Q35','var':'gender','group':['Man'],'group2': 'Woman'}

only_rel= data[[var_0['q'],var_1['q']]].dropna()
# Remove all the text inside the parenthesis and count the number of channels
only_rel[num_to_report] = only_rel[var_0['q']].str.replace(r"\(.*?\)","").str.count(',')
var_0['q'] = num_to_report
calculate_chi_cross_tab(only_rel,var_0,var_1,text_to_update="many")

# For test
# for group in only_rel.groupby('num_to_report'):
#     print(f'{group[0]}:{group[1]["Q11"].unique()}')

Chi-square statistic: 2.08
P-value: 0.14961
gender           Man      Woman
where_rep                      
many       34.512428  45.495495
other      65.487572  54.504505


In [26]:
# Value approx and where to report
var_0 = {'q':'Q13'}
var_1 = {'q':'Q11'}
only_rel= data[[var_0['q'],var_1['q']]].dropna()
pd.crosstab(only_rel[var_1['q']],only_rel[var_0['q']],normalize='columns')*100

Q13,$1000-$1999,$2000-3999,$250-$499,$4000-6999,$500-$999,$7000 or more,Don’t know/not sure,I prefer to not answer,Less than $250
Q11,,,,,,,,,
"A bicycle registry/recovery system (e.g., Bike Index, a university registration program)",2.552204,1.342282,9.016393,0.680272,9.615385,0.000000,0.0,0.0,11.688312
"A bicycle registry/recovery system (e.g., Bike Index, a university registration program),Other (please specify):",0.232019,0.671141,0.409836,0.000000,0.192308,0.000000,0.0,0.0,0.000000
"A bicycle registry/recovery system (e.g., Bike Index, a university registration program),Social media (e.g. Instagram, Twitter)",1.160093,0.671141,4.098361,0.680272,2.500000,2.105263,0.0,0.0,5.194805
"A bicycle registry/recovery system (e.g., Bike Index, a university registration program),Social media (e.g. Instagram, Twitter),Other (please specify):",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.298701
I did not report the theft,1.160093,0.000000,7.786885,0.680272,3.653846,1.052632,0.0,0.0,19.480519
Other (please specify):,1.160093,0.000000,2.049180,0.000000,0.769231,0.000000,0.0,0.0,2.597403
"Social media (e.g. Instagram, Twitter)",0.232019,0.000000,2.459016,0.000000,0.192308,0.000000,0.0,0.0,2.597403
The police,14.385151,17.785235,17.213115,14.965986,16.538462,21.052632,20.0,50.0,24.675325
"The police ,A bicycle registry/recovery system (e.g., Bike Index, a university registration program)",31.786543,30.201342,33.196721,21.768707,30.384615,20.000000,40.0,0.0,18.181818


In [87]:
# age and is_university
var_0 = {'q':'Q17','var':'near_univ','group':['Yes'],'group2': 'No'}
var_1 = {'q':'Q34_0','var':'birth_year','group':['2000s','1990s'],'group2': 'other'}
only_rel= data[[var_0['q'],var_1['q']]].dropna()
clean_tables = only_rel[~only_rel[var_1['q']].isin(['1930s','2010s'])]
calculate_chi_cross_tab(clean_tables,var_0,var_1)

Chi-square statistic: 56.87
P-value: 0.00000
birth_year     1990s     2000s      other
near_univ                                
No          78.39196  45.56962  92.151899
Yes         21.60804  54.43038   7.848101


In [88]:
# age and theft location
var_0 = {'q':'Q9','var':'location','group':['Outdoor bike rack '],'group2': 'other'}
var_1 = {'q':'Q34_0','var':'birth_year','group':['2000s'],'group2': 'other'}
only_rel= data[[var_0['q'],var_1['q']]].dropna()
clean_tables = only_rel[~only_rel[var_1['q']].isin(['1930s','2010s'])]
calculate_chi_cross_tab(clean_tables,var_0,var_1)

Chi-square statistic: 26.82
P-value: 0.00000
birth_year              2000s      other
location                                
Outdoor bike rack   51.898734  16.192283
other               48.101266  83.807717


In [ ]:
# #
# def calculate_chi_cross_tab(two_var_tables,var1,var2,text_to_update = False):
#     r"""
#     This function creates new groups based on the data in @vars, cross tab them and calculate Chi-square
#     :return:
#     """
#     if not text_to_update:
#         two_var_tables[[var1['var'],var2['var']]] = two_var_tables.apply(lambda x: (x[var1['q']] if x[var1['q']] in var1['group'] else var1['group2'],x[var2['q']] if x[var2['q']] in var2['group'] else var2['group2']),axis=1,result_type='expand')
#     else:
#         two_var_tables[[var1['var'],var2['var']]] = two_var_tables.apply(lambda x: (text_to_update if x[var1['q']] in var1['group'] else var1['group2'],x[var2['q']] if x[var2['q']] in var2['group'] else var2['group2']),axis=1,result_type='expand')
#     cont_table = pd.crosstab(two_var_tables[var1['var']], two_var_tables[var2['var']],normalize='columns')*100
#
#     # perform chi-square test of independence
#     chi2, p, dof, expected = chi2_contingency(cont_table)
#
#     # print the test results
#     print(f"Chi-square statistic: {chi2:.2f}")
#     print(f"P-value: {p:.5f}")
#     print(cont_table)